In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from Planet.data import PlanetDataCollection
from Planet.utils import get_transforms
from Planet.train import NNTrainer
from torchvision import models
from pathlib import Path

In [ ]:
path = Path('/home/anukoolpurohit/Documents/AnukoolPurohit/Datasets/Planet')

In [ ]:
tfms = get_transforms()

In [ ]:
data = PlanetDataCollection.from_csv(path, 'train_v2.csv', 'train-jpg',
                           tfms, name_col='image_name', label_col='tags',
                           delimter=' ',bs=64,test_folder='test-jpg')

In [ ]:
data.show()

In [ ]:
def get_df(preds, data, thresh=0.2):
    labeled_preds = [' '.join([data.train_ds.classes[i] for i,p in enumerate(pred) if p > thresh]) for pred in preds]
    fnames = data.test_ds.x.items
    df = pd.DataFrame({'image_name': fnames, 'tags': labeled_preds}, columns=['image_name', 'tags'])
    df['extra'] = df['image_name'].str.split('_', n=1, expand=True)[1]
    df['extra'] = df['extra'].astype(float)
    df.sort_values(by='extra', inplace=True)
    df.drop(columns='extra', inplace=True)
    return df

In [ ]:
arch = models.resnet50

In [ ]:
trainer = NNTrainer(data,arch)

In [ ]:
trainer.train(1e-03,2)

In [ ]:
trainer.train(1e-05,2)

In [ ]:
trainer.save('frozen1')

In [ ]:
trainer.unfreeze()

In [ ]:
trainer.train(1e-05,4)

In [ ]:
trainer.train(1e-06,2)

In [ ]:
trainer.save('done')

In [ ]:
trainer.load('basic_trained')

In [ ]:
preds = trainer.predict_test()

In [ ]:
df = get_df(preds,data)

In [ ]:
data2 = PlanetDataCollection.from_csv(path, 'train_v2.csv', 'train-jpg',
                           tfms, name_col='image_name', label_col='tags',
                           delimter=' ',bs=64,test_folder='test-jpg-additional')

trainer.data = data2

In [ ]:
preds2 = trainer.predict_test()

In [ ]:
df2 = get_df(preds2, data2)

In [ ]:
df3 = pd.concat([df, df2])

In [ ]:
df3.to_csv('strange_submission.csv', index=False)

In [ ]:
print(f"%.6f"% 20)